In [ ]:
# !pip install yfinance tqdm pytz --quiet

In [1]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import pytz

### X 데이터 수집

In [2]:
# SPY_NAV(자산가치) 데이터 가져오기
import pandas as pd

path = "../raw_data/navhist-us-en-spy (1).xlsx"

# 1. 엑셀 불러오기
df = pd.read_excel(path)

# 2. Date를 datetime으로 변환 (날짜가 아닌 건 NaT로 처리)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# 3. Date가 NaT인 행(문장 들어간 행 등) 제거
df = df.dropna(subset=["Date"])

# 4. 형식 'YYYY-MM-DD'로 바꾸기
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# 5. Date, NAV 두 컬럼만 남기기
nav_df = df[["Date", "NAV"]].copy()

# 6. NAV → SPY_NAV로 컬럼명 변경
nav_df = nav_df.rename(columns={"NAV": "SPY_NAV"})

print(nav_df.head())

         Date     SPY_NAV
0  2025-11-18  660.127046
1  2025-11-17  665.567433
2  2025-11-14  671.649115
3  2025-11-13  671.850237
4  2025-11-12  683.129392


In [3]:
nav_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5531 entries, 0 to 5530
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     5531 non-null   object 
 1   SPY_NAV  5531 non-null   float64
dtypes: float64(1), object(1)
memory usage: 129.6+ KB


In [4]:
import pandas as pd

# 1. Date를 datetime으로 변환 (현재 'YYYY-MM-DD' 문자열 상태라고 가정)
nav_df["Date"] = pd.to_datetime(nav_df["Date"])

# 2. 날짜 범위 필터링 (2022-11-08 ~ 2025-11-18 포함)
start = pd.to_datetime("2025-11-10")
end   = pd.to_datetime("2025-11-18")

nav_df = nav_df[(nav_df["Date"] >= start) & (nav_df["Date"] <= end)]

# 3. Date 기준 오름차순 정렬
nav_df = nav_df.sort_values("Date")

# 4. 다시 'YYYY-MM-DD' 문자열로 쓰고 싶으면
nav_df["Date"] = nav_df["Date"].dt.strftime("%Y-%m-%d")

print(nav_df.head())


         Date     SPY_NAV
6  2025-11-10  681.263847
5  2025-11-11  682.675112
4  2025-11-12  683.129392
3  2025-11-13  671.850237
2  2025-11-14  671.649115


In [5]:
nav_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 6 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     7 non-null      object 
 1   SPY_NAV  7 non-null      float64
dtypes: float64(1), object(1)
memory usage: 168.0+ bytes


In [6]:
# ============================================
# 수집할 종목 리스트 정의
# ============================================
symbols = ['SPY']

In [7]:
# =============================
# 2. 시계열 데이터 수집 (뉴욕시간 기준)
# =============================
START = "2025-11-10"
END   = "2025-11-19"

In [8]:
print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

[1/2] 시계열 데이터 수집 중 ...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [ ]:
# UTC로 지정 → 뉴욕시간으로 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# '세션 날짜'를 계산: 전날 저녁 라벨 → 다음날 거래일로 이동
session_dates = (data.index + pd.Timedelta(days=1)).normalize()

# '09:30' (개장시각) 타임스탬프로 교체
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)

# 이걸 인덱스로 지정
data.index = session_times

print("인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료")
print(data.index.min(), "→", data.index.max())

📅 인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료
2025-11-10 09:30:00-05:00 → 2025-11-18 09:30:00-05:00


In [51]:
print("데이터 레벨명:", data.columns.names)   # ['Attributes', 'Symbols'] 형태
print("기간:", data.index.min().date(), "~", data.index.max().date())

데이터 레벨명: ['Price', 'Ticker']
기간: 2025-11-10 ~ 2025-11-18


In [52]:
# 필요한 컬럼 정리
ohlcv = data[["Close"]].copy()
ohlcv.columns = [f"{ticker}_{col}" for col, ticker in ohlcv.columns]

In [53]:
ohlcv

,SPY_Close
Date,
2025-11-10 09:30:00-05:00,681.440002
2025-11-11 09:30:00-05:00,683.000000
2025-11-12 09:30:00-05:00,683.380005
2025-11-13 09:30:00-05:00,672.039978
2025-11-14 09:30:00-05:00,671.929993
2025-11-17 09:30:00-05:00,665.669983
2025-11-18 09:30:00-05:00,660.080017


In [33]:
# ============================================
# 수집할 종목 리스트 정의
# ============================================
symbols = ['^VIX','GC=F','SHY']

In [34]:
# =============================
# 2. 시계열 데이터 수집 (뉴욕시간 기준)
# =============================
START = "2025-11-10"
END   = "2025-11-19"

In [35]:
print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

[*********************100%***********************]  3 of 3 completed

[1/2] 시계열 데이터 수집 중 ...


In [ ]:
# UTC로 지정 → 뉴욕시간으로 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# '세션 날짜'를 계산: 전날 저녁 라벨 → 다음날 거래일로 이동
session_dates = (data.index + pd.Timedelta(days=1)).normalize()

# '09:30' (개장시각) 타임스탬프로 교체
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)

# 이걸 인덱스로 지정
data.index = session_times

print("인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료")
print(data.index.min(), "→", data.index.max())

📅 인덱스(라벨)를 실제 뉴욕 개장 시각(09:30)으로 조정 완료
2025-11-10 09:30:00-05:00 → 2025-11-18 09:30:00-05:00


In [37]:
print("데이터 레벨명:", data.columns.names)   # ['Attributes', 'Symbols'] 형태
print("기간:", data.index.min().date(), "~", data.index.max().date())

데이터 레벨명: ['Price', 'Ticker']
기간: 2025-11-10 ~ 2025-11-18


In [38]:
# 필요한 컬럼 정리
ohlcv = data[["High", "Low","Volume"]].copy()
ohlcv.columns = [f"{ticker}_{col}" for col, ticker in ohlcv.columns]

In [39]:
# 변동성(High-Low)
volatility = pd.DataFrame(index=ohlcv.index)
for sym in symbols:
    h, l = f"{sym}_High", f"{sym}_Low"
    if h in ohlcv.columns and l in ohlcv.columns:
        volatility[f"{sym}_Volatility"] = ohlcv[h] - ohlcv[l]

price_features = pd.concat([ohlcv, volatility], axis=1).sort_index()

In [40]:
price_features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7 entries, 2025-11-10 09:30:00-05:00 to 2025-11-18 09:30:00-05:00
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GC=F_High        7 non-null      float64
 1   SHY_High         7 non-null      float64
 2   ^VIX_High        7 non-null      float64
 3   GC=F_Low         7 non-null      float64
 4   SHY_Low          7 non-null      float64
 5   ^VIX_Low         7 non-null      float64
 6   GC=F_Volume      7 non-null      int64  
 7   SHY_Volume       7 non-null      int64  
 8   ^VIX_Volume      7 non-null      int64  
 9   ^VIX_Volatility  7 non-null      float64
 10  GC=F_Volatility  7 non-null      float64
 11  SHY_Volatility   7 non-null      float64
dtypes: float64(9), int64(3)
memory usage: 728.0 bytes


In [41]:
price_features.columns

Index(['GC=F_High', 'SHY_High', '^VIX_High', 'GC=F_Low', 'SHY_Low', '^VIX_Low',
       'GC=F_Volume', 'SHY_Volume', '^VIX_Volume', '^VIX_Volatility',
       'GC=F_Volatility', 'SHY_Volatility'],
      dtype='object')

In [42]:
price_features = price_features.drop(['GC=F_High', 'SHY_High', '^VIX_High', 'GC=F_Low', 'SHY_Low', '^VIX_Low','SHY_Volume', '^VIX_Volume'], axis=1)

In [43]:
price_features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7 entries, 2025-11-10 09:30:00-05:00 to 2025-11-18 09:30:00-05:00
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GC=F_Volume      7 non-null      int64  
 1   ^VIX_Volatility  7 non-null      float64
 2   GC=F_Volatility  7 non-null      float64
 3   SHY_Volatility   7 non-null      float64
dtypes: float64(3), int64(1)
memory usage: 280.0 bytes


In [44]:
price_features

,GC=F_Volume,^VIX_Volatility,GC=F_Volatility,SHY_Volatility
Date,,,,
2025-11-10 09:30:00-05:00,189,1.219999,51.799805,0.029999
2025-11-11 09:30:00-05:00,569,0.760000,33.300293,0.040001
2025-11-12 09:30:00-05:00,396,0.959999,101.500000,0.030006
2025-11-13 09:30:00-05:00,167,3.799999,65.000000,0.029999
2025-11-14 09:30:00-05:00,483,3.470001,150.899902,0.089996
2025-11-17 09:30:00-05:00,1552,3.900000,80.100098,0.020004
2025-11-18 09:30:00-05:00,1552,2.930000,27.799805,0.070000


### Y데이터 수집

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# ============================================
# 1) 종목 리스트
# ============================================
symbols = ["SPY"]

# ============================================
# 2) 시계열 데이터 수집
# ============================================
START = "2025-11-10"
END   = "2025-11-19"

print("[1/2] 시계열 데이터 수집 중 ...")
data = yf.download(symbols, start=START, end=END, interval="1d", auto_adjust=False)

# UTC → 뉴욕시간 변환
data.index = pd.to_datetime(data.index, utc=True).tz_convert("America/New_York")

# 전날 가격을 다음날 개장시간(09:30)에 반영
session_dates = (data.index + pd.Timedelta(days=1)).normalize()
session_times = session_dates + pd.Timedelta(hours=9, minutes=30)
data.index = session_times

print("NY 개장 시간으로 라벨 조정 완료")
print("기간:", data.index.min().date(), "~", data.index.max().date())

# ============================================
# 3) MultiIndex → 단일 컬럼으로 변환
# ============================================
data.columns = [f"{col}_{sym}" for col, sym in data.columns]

# ============================================
# 4) 필요한 컬럼만 선택
# ============================================
ohlcv = data[["Close_SPY"]].copy()

ohlcv[["Close_SPY"]] = ohlcv[["Close_SPY"]]


# ============================================
# 5) Date 컬럼 생성
# ============================================
ohlcv["Date"] = ohlcv.index.tz_localize(None).normalize().date
ohlcv.reset_index(drop=True, inplace=True)

# Date 컬럼을 맨 앞으로 이동
ohlcv = ohlcv[["Date"] + [col for col in ohlcv.columns if col != "Date"]]

# ============================================
# 6) 출력
# ============================================
print(ohlcv)

[1/2] 시계열 데이터 수집 중 ...


[*********************100%***********************]  1 of 1 completed

NY 개장 시간으로 라벨 조정 완료
기간: 2025-11-10 ~ 2025-11-18
         Date   Close_SPY
0  2025-11-10  683.000000
1  2025-11-11  683.380005
2  2025-11-12  672.039978
3  2025-11-13  671.929993
4  2025-11-14  665.669983
5  2025-11-17  660.080017
6  2025-11-18         NaN


In [46]:
ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       7 non-null      object 
 1   Close_SPY  6 non-null      float64
dtypes: float64(1), object(1)
memory usage: 244.0+ bytes
